## Introduction
Greetings from the Kaggle bot! This is an automatically-generated kernel with starter code demonstrating how to read in the data and begin exploring. If you're inspired to dig deeper, click the blue "Fork Notebook" button at the top of this kernel to begin editing.

## Exploratory Analysis
To begin this exploratory analysis, first import libraries and define functions for plotting the data using `matplotlib`. Depending on the data, not all plots will be made. (Hey, I'm just a simple kerneling bot, not a Kaggle Competitions Grandmaster!)

In [2]:
import pandas as pd
import scipy

In [3]:
data = scipy.io.loadmat('datasets/DREAMER.mat')
print(data.keys())

dict_keys(['__header__', '__version__', '__globals__', 'DREAMER'])


In [4]:
data.keys()

dict_keys(['__header__', '__version__', '__globals__', 'DREAMER'])

In [5]:
data = data['DREAMER']

In [6]:
type(data)

numpy.ndarray

In [7]:
type(data['Data'])

numpy.ndarray

In [8]:
## This is the dict the inluce which feature you want for the patient

Features = {
    'Age' : 0,
    'Gender': 1,
    'EEG' : 2,
    'ECG' : 3,
    'Valence' : 4,
    'Arousal' : 5,
    'Dominance' : 6
}

## This is How you would index any feature you want from the data for any patient 

```python
data['Data'][0,0][0][Sample_index][0][0][Feature][0]
```

Sample_index : int can be any number from 0 to 23 wich is the number of patients
<br>
Feature: Dict for all these values 
```python
Feature = {
    'Age' : 0,
    'Gender': 1,
    'EEG' : 2,
    'ECG' : 3,
    'Valence' : 4,
    'Arousal' : 5,
    'Dominance' : 6
}
```

In [9]:
def retrieve_data(Sample_index, Feature_name):
    """
    Retrieve specific feature data for a given patient from the dataset.

    Parameters:
        Sample_index (int): Index of the patient (0 to 23).
        Feature_name (str): Name of the feature to retrieve. It should be one of the following:
                            'Age', 'Gender', 'EEG', 'ECG', 'Valence', 'Arousal', 'Dominance'.

    Returns:
        float: The value of the specified feature for the specified patient.

    Raises:
        KeyError: If Feature_name is not one of the supported features.
    """
    return data['Data'][0,0][0][Sample_index][0][0][Features[Feature_name]][0]


## Retrieve All the Age for all patients

In [10]:
Age = []

for i in range(23):
    age = retrieve_data(i,'Valence') 
    Age.append(age)
    
Age[0:5]

[array([4], dtype=uint8),
 array([3], dtype=uint8),
 array([3], dtype=uint8),
 array([3], dtype=uint8),
 array([4], dtype=uint8)]

## Retrieve All the Gender for all patients

In [11]:
Gender = []

for i in range(23):
    gender = retrieve_data(i,'Gender')              
    Gender.append(gender)
    
Gender[0:5]

[np.str_('male'),
 np.str_('female'),
 np.str_('male'),
 np.str_('female'),
 np.str_('female')]

## This is how you index EEG and ECG signals

You would still use the retrive EEG and ECG but they are structs so you would Need extra work to acces the data
![image.png](attachment:1be069c7-bb5d-44ff-8463-b95b6a17ff35.png)


I will make a separate Data frame for each feature of EEG and ECG 


In [12]:
signal_types = {
    'BaseLine' :0,
    'Stimulie' :1
}

In [13]:
def retrieve_signals(Sample_index, Feature_type, signal_type):
    """
    Retrieve specific signal data for a given patient from the dataset.

    Parameters:
        Sample_index (int): Index of the patient (0 to 23).
        Feature_type (str): Type of feature ('EEG' or 'ECG').
        signal_type (str): Type of signal to retrieve ('BaseLine' or 'Stimulie').

    Returns:
        float: The value of the specified signal for the specified patient.

    Raises:
        ValueError: If Feature_type is not 'EEG' or 'ECG'.
    """
    if Feature_type not in ('EEG', 'ECG'):
        raise ValueError(f"Invalid Feature_type: {Feature_type}. Feature_type must be 'EEG' or 'ECG'.")

    return data['Data'][0,0][0][Sample_index][0][0][Features[Feature_type]][0][0][signal_types[signal_type]]


## Retrive EEG_Baseline

In [14]:
EEG_Baseline = []

for i in range(23):
    baseline = retrieve_signals(i,'EEG','BaseLine')       
    EEG_Baseline.append(baseline)
    
len(EEG_Baseline[0])

18

## Retrive EEG_Stimlie

In [15]:
EEG_Stimuli = []

for i in range(23):
    stimuli = retrieve_signals(i,'EEG','Stimulie')             
    EEG_Stimuli.append(stimuli)
    
# EEG_Stimuli[0]

## Retrive ECG_Baseline

In [20]:
ECG_Baseline = []

for i in range(23):
    baseline = retrieve_signals(i,'ECG','BaseLine')            
    ECG_Baseline.append(baseline)

# EEG_Baseline[0]

## Retrieve ECG Stimulie

In [19]:
ECG_Stimuli = []

for i in range(23):
    baseline = retrieve_signals(i,'ECG','Stimulie')            
    ECG_Stimuli.append(baseline)
    
len(ECG_Stimuli[0])
valence = [0,0,0,0,0]
arousal = [0,0,0,0,0]
#count valence and arousal
for i in range(23):
    valence[retrieve_data(i,'Valence')[0] - 1] += 1
    arousal[retrieve_data(i,'Arousal')[0] - 1] += 1

print(valence)
print(arousal)




[0, 4, 11, 8, 0]
[3, 12, 7, 1, 0]


## Create a DF that has the patient index and the video index and the EEG reading for the patient. 

![image.png](attachment:854863e6-005b-4567-b933-3591f609187a.png)

In [21]:
big_df = pd.DataFrame()
df = pd.DataFrame()
for i in range(23): # for i in every patient
    for j in range(18): # for j in every Video
        # print("Processing patient", i, "video", j)
        patient_index = [i] * len(EEG_Stimuli[i][j][0])
        video_index = [j] * len(EEG_Stimuli[i][j][0])
        electrodes = []
        for k in range(14): # for k in every electode
            electrode = []
            for l,row  in enumerate(EEG_Stimuli[i][j][0]):
                electrode.append(row[k])
            electrodes.append(electrode)   
            
        Dict = {
            'patient_index' : patient_index,
            'video_index' : video_index,
            'Valence' : [retrieve_data(i,'Valence')] * len(EEG_Stimuli[i][j][0]), 
            'Arousal' : [retrieve_data(i,'Arousal')] * len(EEG_Stimuli[i][j][0])
        }
        columns_list = ['AF3', 'F7', 'F3', 'FC5', 'T7', 'P7', 'O1', 'O2', 'P8', 'T8', 'FC6', 'F4', 'F8', 'AF4']
        for idx , key in enumerate(columns_list):
            Dict[key] = electrodes[idx]
        df = pd.DataFrame(Dict)

        big_df = pd.concat([big_df,df], ignore_index=True)


    


KeyboardInterrupt: 

In [ ]:
big_df.to_csv('datasets/DREAMER.csv', index=False)